In [1]:
import autograd.numpy as np
from autograd import grad
import nlopt, numpy as npf
import matplotlib.pyplot as plt

import sys
sys.path.append('../')

import use_autograd
use_autograd.use = 1
import rcwa
from utils import test_grad
from fft_funs import get_conv

In [6]:
t = 59e-9
Period = 2e-6

lam0=1.2e-6
freq = 1.0

Nx = 50
Ny = 50

In [7]:
nG = 51
# lattice vector
Lx = Period/lam0
Ly = Period/lam0
L1 = [Lx,0.]
L2 = [0.,Ly]

# planewave excitation
p_amp = 0.
s_amp = 1.
p_phase = 0.
s_phase = 0.
# frequency and angles
theta = 0.
phi = 0.

# now consider three layers: vacuum + patterned + vacuum
epsuniform1 = 1.
epsdiff = 11.3+1.0j
epsbkg = 1
epsuniform3 = 1.

thick1 = 1.
thick2 = t/lam0
thick3 = 1.

In [8]:
ctrl = 0
vec = []
def fun_abs(dof,Qabs):

    freqcmp = freq*(1+1j/2/Qabs)
    obj = rcwa.RCWA_obj(nG,L1,L2,freqcmp,theta,phi,verbose=0)
    obj.Add_LayerUniform(thick1,epsuniform1)
    obj.Add_LayerGrid(thick2,epsdiff,epsbkg,Nx,Ny)
    obj.Add_LayerUniform(thick3,epsuniform3)
    obj.Init_Setup(Gmethod=0)

    obj.MakeExcitationPlanewave(p_amp,p_phase,s_amp,s_phase,order = 0)
    obj.GridLayer_getDOF(dof)
        
    Mv = get_conv(1./Nx/Ny,dof.reshape(Nx,Ny),obj.G)
    
    val = obj.Volume_integral(1,Mv,Mv,Mv,normalize=1)
    val = np.real(val)


    if 'autograd' not in str(type(val)):
        global ctrl
        global vec
        vec = npf.copy(dof)
        
        print ctrl,val
        if npf.mod(ctrl,10)==0:
            plt.figure();
            plt.imshow(np.reshape(dof,(Nx,Ny)))
            plt.colorbar()
            plt.show()
            
        ctrl +=1
    return val

In [ ]:
Qabs = 100.
fun = lambda dof: fun_abs(dof,Qabs)
grad_fun = grad(fun)
def fun_nlopt(dof,gradn):
    gradn[:] = grad_fun(dof)
    return fun(dof)

ndof = Nx*Ny
#init = np.zeros(ndof)+1e-1*np.random.random(ndof)
init = np.random.random(ndof)
lb=np.zeros(ndof,dtype=float)
ub=np.ones(ndof,dtype=float)

opt = nlopt.opt(nlopt.LD_MMA, ndof)
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)

opt.set_ftol_rel(1e-10)
opt.set_maxeval(500)

opt.set_min_objective(fun_nlopt)
x = opt.optimize(init)

In [ ]:
np.mean(vec)

In [ ]:
Qabs = 10.
fun = lambda dof: fun_abs(dof,Qabs)
grad_fun = grad(fun)

x = np.random.random(Nx*Ny)
test_grad(fun,grad_fun,x,1e-5,0)